In [1]:
import pandas as pd
import numpy as np

/var/folders/yv/mzlh10ts71lg0nhr7pw_0q080000gn/T/ipykernel_46206/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('/Users/catherinebreen/Dropbox/CBRE and SnowEx/SnowEx_for_NSIDC/2020_camera_imagery_submission/GM_snowex2020_timelapsecams.csv')
cameras = pd.unique([i.split(' ')[0] for i in data['ident']])
print(cameras)
data['ID'] = [i.split(' ')[0] for i in data['ident']]
data['type'] = [i.split(' ')[1] for i in data['ident']]
print(data.head())

['E3A' 'E6A' 'E6B' 'E8A' 'E9A' 'E9B' 'E9C' 'E9D' 'E9E' 'E9F' 'E9G'
 'TLSK20' 'W1A' 'W1B' 'W2A' 'W2B' 'W3A' 'W5A' 'W6A' 'W6B' 'W6C' 'W8A'
 'W8B' 'W8C' 'W9A' 'W9B' 'W9C' 'W9D' 'W9E' 'W9G']
    ID     ident   Latitude   Longitude     y_UTM12N     x_UTM12N  type
0  E3A   E3A CAM  39.108011 -107.881267  4333394.936  769672.9616   CAM
1  E3A  E3A POLE  39.107937 -107.881191  4333386.947  769679.8169  POLE
2  E6A   E6A CAM  39.097489 -107.862526  4332282.791  771334.1861   CAM
3  E6A  E6A POLE  39.097464 -107.862476  4332280.166  771338.6070  POLE
4  E6B   E6B CAM  39.048970 -107.913070  4326746.985  767145.2541   CAM


/var/folders/yv/mzlh10ts71lg0nhr7pw_0q080000gn/T/ipykernel_46206/477144364.py:2: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  cameras = pd.unique([i.split(' ')[0] for i in data['ident']])


In [3]:
# Function to calculate the Euclidean distance between two points in UTM coordinates
def calculate_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)


In [4]:
# Function to calculate the Haversine distance between two points in latitude and longitude
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in meters
    R = 6371000
    # Convert latitude and longitude from degrees to radians
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    # Haversine formula
    a = np.sin(delta_phi / 2.0)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c


In [5]:
dist = []
for cam in cameras: 
    df = data[data['ID'] == cam]
    c = df[df['type'] == 'CAM'].reset_index()
    p = df[df['type'] == 'POLE'].reset_index()
    m = calculate_distance(c['x_UTM12N'][0].tolist(), c['y_UTM12N'][0].tolist(), p['x_UTM12N'][0].tolist(), p['y_UTM12N'][0].tolist())
    #m = haversine(c['Latitude'][0].tolist(), c['Longitude'][0].tolist(), p['Latitude'][0].tolist(), p['Longitude'][0].tolist())
    dist.append(m)

In [6]:
results = pd.DataFrame({'cam': cameras, 'dist': dist})

In [7]:
results.head()

,cam,dist
0,E3A,10.527073
1,E6A,5.141496
2,E6B,11.165701
3,E8A,10.503074
4,E9A,10.522233


In [8]:
results.to_csv('/Users/catherinebreen/Dropbox/Chapter1/WRRsubmission/distance_from_cam.csv')